In [1]:
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.PINN import PINN
from methods.methodsDataset.PINNDataset import PINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1e-3, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [4]:
def delta(y, x, dy = 1., dx = 0.) :
    """    
    y : int, float or ndarray of size 1
    x : ndarray
    
    return dy if x = y and dx otherwise
    """
    if torch.is_tensor(y) or torch.is_tensor(y):
        return torch.where(x == y, dy, dx)
    return np.where(x == y, dy, dx)

In [5]:
params_solver['source_term'] = lambda y, x : delta(y, x)
params_solver['y'] = None

In [6]:
solver = Solver(params={'solver': params_solver})

In [7]:
Y_list = solver.x[20:81].reshape(-1, 1)

In [8]:
U_sols = []
for y in Y_list:
    solver.change_y(new_y= y)
    U_sols.append(solver.solve())
U_sols = np.stack(U_sols)

In [9]:
Y_train, Y_val, U_train, U_val = sklearn.model_selection.train_test_split(Y_list, 
                                                                          U_sols, 
                                                                          test_size=0.2, 
                                                                          random_state=123)

In [10]:
nx = params_solver['nx']

x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(Y_train.shape[0], 1)
X_val = x.repeat(Y_val.shape[0], 1)


Y_train_repeated = torch.Tensor(Y_train.repeat(nx, 1).reshape(-1, 1))
Y_val_repeated = torch.Tensor(Y_val.repeat(nx, 1).reshape(-1, 1))


YX_train = torch.cat((Y_train_repeated, X_train), dim=1)
YX_val = torch.cat((Y_val_repeated, X_val), dim=1)
YU_train = torch.Tensor(U_train).view(-1, 1)
YU_val = torch.Tensor(U_val).view(-1, 1)

In [11]:
def get_dataset():
    return YX_train, YX_val, YU_train, YU_val

In [12]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [13]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 8, 40)
        layers += [out_features]
        
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = 'tanh'
    return PINN(params={'solver':params_solver, 'method':params_PINN_trial})

In [14]:
YX_train, YX_val, YU_train, YU_val = get_dataset()

In [15]:
def objective(trial):
    # Generate the model.
    
    model = define_model(trial, 2, 1)
    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    
    YX_train, YX_val, YU_train, YU_val = get_dataset()
    
    torch.manual_seed(params_PINN['seed'])
    
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    # Training of the model.
    
    w_r = trial.suggest_float("w_r", 1e-1, 1e+3, log=True)
    model.fit(hyperparameters = hyperparameters, 
              DX_train = YX_train, DX_val = YX_val, U_val = YU_val, U_train = YU_train,
              physics_ratio = 1.,
              data_ratio = 0.,
              loss='Y',
              w_r = w_r)
    
    #V = model.loss_dict['val']
    R = model.loss_dict['train']['residual']
    B = model.loss_dict['train']['ic_bc']
    
    S = [r + b for r, b in zip(R, B)]
    m = min(S)
    
    trial.report(m, n_epochs)

    return m

In [16]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-16 09:22:41,374] A new study created in memory with name: no-name-5eae9b47-5687-43e9-ab77-6261ed4a653c


In [17]:
study.optimize(objective, n_trials=10)

[tr : 2.3e-03, val : 5.6e-03]: 100%|█████| 10000/10000 [00:45<00:00, 221.88it/s]
[I 2023-08-16 09:23:27,160] Trial 0 finished with value: 0.00013307697895470483 and parameters: {'n_layers': 4, 'units_l0': 17, 'units_l1': 15, 'units_l2': 26, 'units_l3': 31, 'lr': 0.00018591711878786343, 'w_r': 837.6388146302431}. Best is trial 0 with value: 0.00013307697895470483.
[tr : 1.7e-03, val : 1.7e+00]: 100%|█████| 10000/10000 [00:43<00:00, 231.13it/s]
[I 2023-08-16 09:24:10,430] Trial 1 finished with value: 0.009899365440094066 and parameters: {'n_layers': 4, 'units_l0': 23, 'units_l1': 20, 'units_l2': 19, 'units_l3': 32, 'lr': 0.00020687915518060283, 'w_r': 0.17326529663635618}. Best is trial 0 with value: 0.00013307697895470483.
[tr : 1.3e-01, val : 1.7e+00]: 100%|█████| 10000/10000 [00:26<00:00, 376.31it/s]
[I 2023-08-16 09:24:37,007] Trial 2 finished with value: 0.009901670358431147 and parameters: {'n_layers': 2, 'units_l0': 32, 'units_l1': 14, 'lr': 3.360123720029353e-05, 'w_r': 13.372281

In [18]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10
Best trial:
  Value:  3.3803175483626546e-05
  Params: 
    n_layers: 2
    units_l0: 21
    units_l1: 30
    lr: 0.00423029374725911
    w_r: 11.007516621231435


In [19]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'w_r':
        w_r = value

Use this to fill config_step_1.py

In [20]:
print(activations, '\n', layer_dims, '\n', lr, '\n', w_r)

[] 
 [21, 30] 
 0.00423029374725911 
 11.007516621231435
